<a id='TOC'></a>

# Cleaning Data (Part 1)
The purpose of this notebook is to read in raw excel data for multiple years, append them into a single dataframe.

# [Resume Here](#ResumeHere)

# Table of Contents
1. [Setting up Python](#SettingUp)
    
    a. [Setting the Location](#SettingLoc)
    
    b. [Importing Necessary Packages](#ImportingPackages)
    
    c. [Preparing for a Save](#PreparingSave)
    

2. [Handling Columns](#HandlingColumns)
    
    a. [Find Unique Column Names](#FindUniqueCol)
    
    b. [Eliminate Unnecessary Columns](#DropCol)
    
    c. [Combine Synonyms](#CombineCol)

3. [Reading and Appending Data](#ReadingAppendingData)

4. [Exporting Data](#ExportingData)

<a id='SettingUp'></a>

# Setting up Python
[Top](#TOC)

[Setting the Location](#SettingLoc)
    
[Importing Necessary Packages](#ImportingPackages)
    
[Getting Data](#GettingData)
    
[Preparing for a Save](#PreparingSave)

<a id='ImportingPackages'></a>

## Importing Necessary Packages

[Top](#TOC)

[Setting Up Python](#SettingUp)

Here we import necessary packages. 
This chunk may take a while.

In [1]:
import pandas as pd
import numpy as np
import glob,os
from liz_number import lizsort,mindate,smallest,validate
from liz_toes import make_str,label_pattern, replace_pattern,report_pattern

import plotly
import plotly.plotly as py
import plotly.graph_objs as go

plotly.tools.set_config_file(world_readable=True)

# increase print limit
pd.options.display.max_rows = 99999

<a id='PreparingSave'></a>

## Preparing for a Save
[Top](#TOC)

[Setting up Python](#SettingUp)

<a id='SettingLoc'></a>

## Setting the Location
[Top](#TOC)

[Setting Up Python](#SettingUp)

These chunks identify the locations from which we can get data and to which we can save data.

### Source Data
Raw data can be found in the following locations:

In [2]:
# sourceDataPers = 'C:/Users/Christopher/Google Drive/TailDemography/outputFiles'
sourceDataBig = 'S:/Chris/TailDemography/TailDemography/Raw Data'
# sourceBlack = 'C:/Users/test/Desktop'

Now we change the working directory to the source path.

In [3]:
os.chdir(sourceDataBig)

### Output Data
The cleaned data will be saved to one of these locations:

In [4]:
# outputPers = 'C:/Users/Christopher/Google Drive/TailDemography/outputFiles'
outputBig = 'S:/Chris/TailDemography/TailDemography/Cleaned Combined Data'
# outputBlack = 'C:/Users/test/Desktop'

<a id='HandlingColumns'></a>

# Handling Columns
[Top](#TOC)

We don't have to look in the multiple-sheet file.  It's clear that we'll have to identify a common set of columns prior to combining these files.  Let's define a few functions to help us do this.

We will want to do the following:
1. [Find Unique Column Names](#FindUniqueCol)
2. [Eliminate Unnecessary Columns](#DropCol)
3. [Combine Synonyms](#CombineCol)

Here we use search the source path to locate and eventually read the raw data into our notebook.

In [5]:
rawfiles = glob.glob('*.xls*')
rawfiles

['CC 2000-03-modified from CC-SJ 00-03 final.xls',
 'CC 2004.xlsx',
 'CC 2015 - captures.xls',
 'CC 2016 - captures.xls',
 'CC 2017 Lizards - 3viii17 captures and obs.xls',
 'xCC2005x.xls',
 'xCC2006x.xls',
 'xCC2007x.xls',
 'xCC2008x.xls',
 'xCC2009x.xls',
 'xCC2010x.xlsx',
 'xCC2011x.xls',
 'xCC2012x.xls',
 'xCC2013x.xls',
 'xCC2014x.xlsx']

Let's inspect these data.  We will begin by looking at the number of columns and rows. In each file. To do this first we define a function, *xlsheets*, which we define below.

In [16]:
def xlsheets(file):
    """The function takes a string, *file*, and returns the number and names of sheets in that file.\
    The function relies on the pandas package."""
    tmp = pd.ExcelFile(file)
    res = {'file_name':file,'num_sheets':len(tmp.sheet_names),'names': tmp.sheet_names}
    return res

You can see that when we call *xlsheets* on the first file in our list of files, it returns the total number of sheets and names of each sheet.

In [17]:
xlsheets(rawfiles[0])

{'file_name': 'CC 2000-03-modified from CC-SJ 00-03 final.xls',
 'num_sheets': 4,
 'names': ['2000', '2001', '2002', '2003']}

We will apply this to all of the files in raw files to inspect the data at the highest level.

In [22]:
filelayout = pd.Series(rawfiles).apply(xlsheets)
filelayout

0     {'file_name': 'CC 2000-03-modified from CC-SJ ...
1     {'file_name': 'CC 2004.xlsx', 'num_sheets': 1,...
2     {'file_name': 'CC 2015 - captures.xls', 'num_s...
3     {'file_name': 'CC 2016 - captures.xls', 'num_s...
4     {'file_name': 'CC 2017 Lizards - 3viii17 captu...
5     {'file_name': 'xCC2005x.xls', 'num_sheets': 1,...
6     {'file_name': 'xCC2006x.xls', 'num_sheets': 1,...
7     {'file_name': 'xCC2007x.xls', 'num_sheets': 2,...
8     {'file_name': 'xCC2008x.xls', 'num_sheets': 1,...
9     {'file_name': 'xCC2009x.xls', 'num_sheets': 1,...
10    {'file_name': 'xCC2010x.xlsx', 'num_sheets': 1...
11    {'file_name': 'xCC2011x.xls', 'num_sheets': 1,...
12    {'file_name': 'xCC2012x.xls', 'num_sheets': 1,...
13    {'file_name': 'xCC2013x.xls', 'num_sheets': 1,...
14    {'file_name': 'xCC2014x.xlsx', 'num_sheets': 1...
dtype: object

In [23]:
filelayout = pd.DataFrame([list(file.values()) for file in filelayout])
filelayout.columns = ['file_name','num_sheets','sheetnames']
filelayout

,file_name,num_sheets,sheetnames
0,CC 2000-03-modified from CC-SJ 00-03 final.xls,4,"[2000, 2001, 2002, 2003]"
1,CC 2004.xlsx,1,[2004 ]
2,CC 2015 - captures.xls,1,[2015]
3,CC 2016 - captures.xls,1,[2016]
4,CC 2017 Lizards - 3viii17 captures and obs.xls,1,[2017]
5,xCC2005x.xls,1,[2005]
6,xCC2006x.xls,1,[2006]
7,xCC2007x.xls,2,"[Sheet1, 2007]"
8,xCC2008x.xls,1,[2008]
9,xCC2009x.xls,1,[2009]


We will need to treat the 1st and 8th files in the source data differently when we read in the data.  Let's group the file names accordingly.

In [25]:
rawfiles_ms = [rawfiles[0],rawfiles[7]]
rawfiles_ss = list(set(rawfiles)- set(rawfiles_ms))

The names of files with multiple sheets are now in the variable *rawfiles_ms*.

In [26]:
rawfiles_ms

['CC 2000-03-modified from CC-SJ 00-03 final.xls', 'xCC2007x.xls']

The names of files with a single sheet are now in the variable *rawfiles_ss*.

In [27]:
rawfiles_ss

['CC 2015 - captures.xls',
 'xCC2006x.xls',
 'CC 2016 - captures.xls',
 'xCC2013x.xls',
 'xCC2005x.xls',
 'CC 2017 Lizards - 3viii17 captures and obs.xls',
 'xCC2009x.xls',
 'xCC2014x.xlsx',
 'xCC2008x.xls',
 'CC 2004.xlsx',
 'xCC2010x.xlsx',
 'xCC2011x.xls',
 'xCC2012x.xls']

Now let's take a look at the number of columns in each file. We'll start with the single sheet files, since this is the easiest.  We will define another function, *xlcolshape* to make this easier.

In [28]:
def xlcolshape(file):
    """xlcolshape takes a file name as a string and returns the shape of the excel file"""
    return pd.read_excel(file).shape

When we call this function on the first of the single-sheet files, we can see that it returns a tuple in the format ('number of rows', 'number of columns').

In [29]:
xlcolshape(rawfiles_ss[0])

(241, 19)

We will apply this funtion to the list of single-sheet files for our inspection.

In [30]:
pd.Series(rawfiles_ss).apply(xlcolshape)

0     (241, 19)
1     (163, 16)
2     (103, 21)
3     (106, 19)
4     (202, 16)
5     (798, 21)
6     (162, 16)
7      (97, 19)
8     (134, 20)
9     (479, 16)
10     (99, 41)
11     (64, 19)
12     (85, 19)
dtype: object

<a id='FindUniqueCol'></a>

## Finding Unique Columns
[Top](#TOC)

[Handling Columns](#HandlingColumns)

We'll define a function to extract column names and convert them to an approved set.  We'll use that function to allow us to only add unique names to a list of column names.

In [31]:
def xluniquecol(file,sheet=0):
    """xluniquecol takes a string filename of an excel file and extracts the column names as a set."""
    return list(pd.read_excel(file,sheet_name=sheet).columns)


Here is an example of how xluniquecol works.

In [32]:
xluniquecol(rawfiles_ss[0])

['Species',
 'Toes',
 'Date',
 'Sex',
 'SVL',
 'TL',
 'RTL',
 'Mass',
 'Paint Mark',
 'Location',
 'Meters',
 'New/Recap',
 'Painted',
 'Misc.',
 'Vial',
 'Time',
 'Unnamed: 16',
 'Spotted',
 'Mark']

Now we will create an empty set, *uniquecols*, that will eventually contain the unique column names in all of the files.

We will append the unique column names from each file to *uniquecols*.

In [33]:
tmp = pd.Series(rawfiles_ss).apply(xluniquecol)
uniquecols = list()
for u in tmp:
    uniquecols = uniquecols+u
uniquecols = list(set(uniquecols))
uniquecols

[1,
 'Toe 12',
 'sex',
 'painted or not',
 'Unnamed: 17',
 'Vial',
 'paint mark',
 'RTL',
 'Unnamed: 19',
 'TL (mm)',
 '2015 or earlier',
 'meters',
 'Misc.',
 'RTL (mm)',
 'misc',
 'Date',
 'species',
 'Toe 3',
 'Painted',
 'Toe 20',
 'Unnamed: 16',
 'Toe 9',
 'Year',
 'mass (g)',
 'Toe 1',
 'Toe 2',
 'misc/notes',
 'New/Recap',
 'mass',
 'NEW/recap',
 'Mass',
 'Marked',
 'Toe 5',
 'Toe 11',
 'Toe 15',
 'Sex',
 'Tail condition (1=intact; 2=autotomized; 3=regrown)',
 'Toe 14',
 'Toe 4',
 'Species',
 'date',
 ' painted or not',
 'Collectors',
 'TL',
 'Toe 6',
 'TIME',
 'Toes',
 'SVL',
 'Spotted',
 'location',
 'painted',
 'VIAL',
 'Toe 13',
 'Toe 16',
 'Mark',
 'Toe 19',
 'Paint Mark',
 'TOES',
 'Toe 10',
 'Years Alive (known)',
 'Toe 8',
 'Toe 7',
 'Meters',
 'SVL (mm)',
 'Toe 17',
 'Toe 18',
 'Location',
 'Time',
 '1st Capture (year)']

We will use a similar logic but loop within each file.

In [34]:
help(xluniquecol)

Help on function xluniquecol in module __main__:

xluniquecol(file, sheet=0)
    xluniquecol takes a string filename of an excel file and extracts the column names as a set.



In [81]:
int(filelayout.loc[filelayout.file_name==tmpfile].num_sheets)

4

In [97]:
for file in filelayout.file_name:
    tmpfile = file
    tmprange = range(0,(int(filelayout.loc[filelayout.file_name==tmpfile].num_sheets)-1))
    tmpsheet=filelayout.loc[filelayout.file_name==tmpfile,'sheetnames']
    print(i for i in tmprange)

<generator object <genexpr> at 0x00000281B65F6830>
<generator object <genexpr> at 0x00000281B65F6830>
<generator object <genexpr> at 0x00000281B65F6830>
<generator object <genexpr> at 0x00000281B65F6830>
<generator object <genexpr> at 0x00000281B65F6830>
<generator object <genexpr> at 0x00000281B65F6830>
<generator object <genexpr> at 0x00000281B65F6830>
<generator object <genexpr> at 0x00000281B65F6830>
<generator object <genexpr> at 0x00000281B65F6830>
<generator object <genexpr> at 0x00000281B65F6830>
<generator object <genexpr> at 0x00000281B65F6830>
<generator object <genexpr> at 0x00000281B65F6830>
<generator object <genexpr> at 0x00000281B65DDFC0>
<generator object <genexpr> at 0x00000281B65F6830>
<generator object <genexpr> at 0x00000281B65DDFC0>


In [64]:
xluniquecol(filelayout.file_name[0],sheet=filelayout.sheetnames[0][0])

['sj',
 ' 1-13-19',
 datetime.datetime(2000, 3, 17, 0, 0),
 'f',
 52,
 74,
 0,
 4.2,
 'r1c',
 '1falls',
 'NEW',
 'Unnamed: 11',
 'Unnamed: 12']

In [ ]:
tmp2 = pd.Series(rawfiles_ms).apply(xluniquecol)
uniquecols = list()
for u in tmp:
    uniquecols = uniquecols+u
uniquecols = list(set(uniquecols))
uniquecols

<a id='DropCol'></a>

## Eliminate Unnecessary Columns
[Top](#TOC)

[Cleaning Data](#CleaningData)

[Handling Columns](#HandlingColumns)

Now we will try to identify unnecessary columns and eliminate them. Much of this will be done manually.

In [ ]:
keepCol = ['species', 'date', 'sex', 'svl', 'tl', 'rtl', 'mass',
       'paint.mark', 'location', 'meters', 'new.recap', 'painted', 'misc',
       'vial', 'autotomized', 'sighting', 'toes']

In [ ]:
set(pd.Series(keepCol).str.lower())-set(pd.Series(uniquecols).str.lower())

In [ ]:
set(pd.Series(uniquecols).str.lower())-set(pd.Series(keepCol).str.lower())

Since data for years 2000-2003 are contained in the same Excel file we will have to treat this file differently than the others.

<a id='CombineCol'></a>

## Combining Synonymous Columns
[Top](#TOC)

[Cleaning Data](#CleaningData)

[Handling Columns](#HandlingColumns)

Once we have identified the columns we need to keep, we'll need to apply this list to the files as they are read into python by doing the following:

Let's define a function to identify potential synonyms. One approach is to take a column name that we want to keep and search through a list of names to identify any that contain enough of that name to be considered potentially synonymous.  Let's try to return these potential matches as values in a dictionary and return the desired name as the key for those values.

In [ ]:
def colmatchtodict(x,series, dictsource, key= None):
    """This takes a string, x, and a looks for values in a series that match that contain that string.
    Those values which match are returned as values in a python dict for the key, key."""
    
    assert isinstance(series,pd.Series)
    if key is None:
        key = x
    tmp = series[series.astype(str).str.contains(x,case = False)].tolist()
    dictsource[key] = tmp
    return dictsource
    

In [ ]:
coldict = {}

In [ ]:
colmatchtodict('toes',pd.Series(uniquecols),coldict, key = 'toes')

Now let's see what happened when we apply this funtion to our, keepCol.

In [ ]:
coldict = {}

In [ ]:
pd.Series(keepCol).apply(lambda x: colmatchtodict(x=x,series=pd.Series(uniquecols),dictsource=coldict))
coldict

We will manually adjust the values for 'tl'.

In [ ]:
coldict['tl']=['TL (mm)', 'TL', 'tl']

Now we need to use this dict to relabel the columns we wish to keep.

We need a function that will take a value in *uniquecols*

In [ ]:
def findsyn (name,dictionary, verbose = True):
    """
    *findsyn* checks searches the values of the dict *dictionary* for the string, *name* and returns 
    the key for the key,value pair to which *name* belongs.
    """
    tmp = pd.DataFrame({'preferredcol':list(dictionary.keys()),'synonymns':list(dictionary.values())})
    try:
        res = list(tmp.preferredcol[tmp.synonymns.apply(lambda x:name in x)])[0]
    except:
        res = None
        if verbose == True:
            print("No value matching \"{}\" was found in the dictionary.".format(name))
    return res

Here is are a few examples of how *findsyn* works.

In [ ]:
findsyn('RTi',coldict,verbose=False)

In [ ]:
findsyn('RTi',coldict,verbose=True)

In [ ]:
findsyn('RTL',coldict,verbose=True)

Now we apply *findsyn* to *uniquecol* and create a column of synonyms.

In [ ]:
uniquecolsdf = pd.DataFrame({'uniquecols':uniquecols})
uniquecolsdf['preferredcol'] = uniquecolsdf.uniquecols.apply(lambda x: findsyn(x,coldict,False))
uniquecolsdf

Now we will turn this dataframe back into a dict so that we can easily use it to rename columns

In [ ]:
uniquecolsdf.index = uniquecolsdf.uniquecols
uniquecolsdict = pd.Series(uniquecolsdf.preferredcol).to_dict()
uniquecolsdict

We'll use the dict, *uniquecolsdict* to rename the synonymous columns in our file....once we read them in,
that is.

<a id='ReadingAppendingData'></a>

# Reading and Appending Data
[Top](#TOC)

In [ ]:
def readnclean(x,dictionary,dtype=None):
    """
    This function reads an excel file, renames columns deemed synonymous according to a dict,
    *dictionary*, and drops unnecessary columns before returning the cleaner dataframe.
    """
    tmp = pd.read_excel(x,dtype=dtype)
    tmp.columns = pd.Series(tmp.columns).map(lambda x:dictionary[x])
    dropidx =[None==col for col in list(tmp.columns)]
    tmp=tmp.drop(columns=tmp.columns[dropidx])
    
    return tmp

Here is an example of how *readnclean* works.

In [ ]:
readnclean(rawfiles_ss[0],uniquecolsdict,dtype=str).head()

We'll create a df, *df*, with no data, but columns from our desired columns, *i.e.* the keys for coldict, as a placeholder to which we can append new data.

In [ ]:
df = pd.DataFrame(columns=coldict.keys())
df

Now we will read in all of the raw_ss files clean the column names and concatenate them into one large df.

In [ ]:
for file in rawfiles_ss:
    df = pd.concat([df,readnclean(file,uniquecolsdict)],sort=True)
    print(df.shape[0])
print("\n\nFinal df has {} columns and {} rows.".format(df.shape[1],df.shape[0]))
df.head()

For the *rawdata_ms* files we will apply a similar logic

<a id='ExportingData'></a>

# Exporting Data
[Top](#TOC)